In [2]:
import pandas as pd

In [4]:
df = pd.read_csv(
    "../datasets/local/vis_papers/raw/IEEE VIS papers 1990-2022 - Main dataset.csv"
)

In [5]:
df

,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount,CitationCount_CrossRef,PubsCited_CrossRef,Award
0,Vis,2022,Photosensitive Accessibility for Interactive D...,10.1109/TVCG.2022.3209359,http://dx.doi.org/10.1109/TVCG.2022.3209359,374,384,J,Accessibility guidelines place restrictions on...,Laura South;Michelle Borkin,Laura South;Michelle A. Borkin,"Northeastern University, USA;Northeastern Univ...",10.1109/TVCG.2011.185;10.1109/TVCG.2021.311482...,"accessibility,photosensitive epilepsy,photosen...",NaN,1.0,63.0,NaN
1,Vis,2022,HetVis: A Visual Analysis Approach for Identif...,10.1109/TVCG.2022.3209347,http://dx.doi.org/10.1109/TVCG.2022.3209347,310,319,J,Horizontal federated learning (HFL) enables di...,Xumeng Wang;Wei Chen 0001;Jiazhi Xia;Zhen Wen;...,Xumeng Wang;Wei Chen;Jiazhi Xia;Zhen Wen;Rongc...,"TMCC, CS, Nankai University, China;State Key L...",10.1109/TVCG.2015.2467618;10.1109/TVCG.2019.29...,"Federated learning,data heterogeneity,cluster ...",NaN,3.0,43.0,NaN
2,Vis,2022,Rigel: Transforming Tabular Data by Declarativ...,10.1109/TVCG.2022.3209385,http://dx.doi.org/10.1109/TVCG.2022.3209385,128,138,J,"We present Rigel, an interactive system for ra...",Ran Chen;Di Weng;Yanwei Huang;Xinhuan Shu;Jiay...,Ran Chen;Di Weng;Yanwei Huang;Xinhuan Shu;Jiay...,"State Key Lab of CAD&CG, Zhejiang University, ...",10.1109/TVCG.2021.3114830;10.1109/VAST47406.20...,"Data transformation,self-service data transfor...",NaN,3.0,68.0,NaN
3,Vis,2022,BeauVis: A Validated Scale for Measuring the A...,10.1109/TVCG.2022.3209390,http://dx.doi.org/10.1109/TVCG.2022.3209390,363,373,J,We developed and validated a rating scale to a...,Tingying He;Petra Isenberg;Raimund Dachselt;To...,Tingying He;Petra Isenberg;Raimund Dachselt;To...,"Université Paris-Saclay, CNRS, Inria, LISN, Fr...",10.1109/INFVIS.2005.1532128;10.1109/TVCG.2006....,"Aesthetics,aesthetic pleasure,validated scale,...",NaN,1.0,79.0,NaN
4,Vis,2022,NAS-Navigator: Visual Steering for Explainable...,10.1109/TVCG.2022.3209361,http://dx.doi.org/10.1109/TVCG.2022.3209361,299,309,J,The success of DL can be attributed to hours o...,Anjul Tyagi;Cong Xie;Klaus Mueller 0001,Anjul Tyagi;Cong Xie;Klaus Mueller,"Computer Science Department, Visual Analytics ...",10.1109/VAST.2012.6400490;10.1109/TVCG.2019.29...,"Deep Learning,Neural Network Architecture Sear...",NaN,0.0,63.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,Vis,1990,Visualization and three-dimensional image proc...,10.1109/VISUAL.1990.146376,http://dx.doi.org/10.1109/VISUAL.1990.146376,144,"149, 469",C,The author applied image processing and volume...,Nahum D. Gershon,N.D. Gershon,"MITRE Corporation, Mclean, VA, USA",NaN,NaN,10.0,5.0,5.0,NaN
3616,Vis,1990,Case study in scientific visualization: factor...,10.1109/VISUAL.1990.146415,http://dx.doi.org/10.1109/VISUAL.1990.146415,430,434,C,The problem of presenting and gaining deeper u...,Wayne E. Fordyce;Jeffrey Ventrella,W.E. Fordyce;J.J. Ventrella,"Research Computing Services, Syracuse Universi...",NaN,NaN,1.0,1.0,10.0,NaN
3617,Vis,1990,Hierarchical triangulation using terrain features,10.1109/VISUAL.1990.146379,http://dx.doi.org/10.1109/VISUAL.1990.146379,168,175,C,A hierarchical triangulation built from a digi...,Lori L. Scarlatos;Theodosios Pavlidis,L. Scarlatos;T. Pavlidis,"Grumman Data Systems, Woodbury, NY, USA;Depart...",NaN,NaN,43.0,7.0,20.0,NaN
3618,Vis,1990,A methodology for scientific data visualisatio...,10.1109/VISUAL.1990.146372,http://dx.doi.org/10.1109/VISUAL.1990.146372,114,123,C,A methodology for guiding the choice of visual...,Philip K. Robertson,P.K. Robertson,"Centre for Spatial Information Systems, Divisi...",NaN,NaN,4.0,13.0,21.0,NaN


In [22]:
abstracts = df["Abstract"]
abstracts_non_null = abstracts.dropna().reset_index(drop=True)

In [4]:
from textprofilerbackend.transform import get_word_tokens_batch, get_byte_encoding_batch
from textprofilerbackend.textclean import get_textcol_metadata_embeddings

/Users/wepperso/miniconda3/envs/textprofiler/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_names = [
    "all-mpnet-base-v2",  # best performing on leaderboard
    # "all-MiniLM-L6-v2",  # smaller and faster
]

In [26]:
meta_data_dict = get_textcol_metadata_embeddings(abstracts_non_null, model_names)

Created embedding of shape torch.Size([3549, 768]) with all-mpnet-base-v2


In [27]:
meta_data_dict["heuristic_metadata"]

,Abstract_text_length,Abstract_num_words,Abstract_max_word_length,Abstract_avg_word_length,Abstract_perc_special_chars
0,2109,286,22,6.377622,0.016121
1,1153,167,14,5.910180,0.019948
2,1543,220,16,6.018182,0.017498
3,1472,223,15,5.605381,0.025136
4,1879,272,17,5.911765,0.027674
...,...,...,...,...,...
3544,758,108,28,6.027778,0.026385
3545,786,111,29,6.090090,0.036896
3546,898,138,25,5.514493,0.026726
3547,879,116,26,6.586207,0.018203


In [28]:
meta_data_dict["model_embeddings"]["all-mpnet-base-v2"]

tensor([[-0.0186,  0.0270, -0.0430,  ...,  0.0387, -0.0176, -0.0138],
        [-0.0256,  0.0685, -0.0391,  ..., -0.0113, -0.0013,  0.0081],
        [-0.0303, -0.0032, -0.0174,  ...,  0.0170, -0.0464, -0.0170],
        ...,
        [-0.0179, -0.0226, -0.0176,  ..., -0.0003,  0.0411,  0.0018],
        [-0.0503,  0.0167, -0.0140,  ...,  0.0455, -0.0256, -0.0077],
        [-0.0025,  0.0178, -0.0276,  ...,  0.0423, -0.0274,  0.0337]])

In [29]:
words = get_word_tokens_batch(abstracts_non_null)

In [30]:
tokens = get_byte_encoding_batch(abstracts_non_null)

In [33]:
df_non_null = df[df.Abstract.notnull()]

In [38]:
df_non_null = df_non_null.reset_index(drop=True)

In [41]:
df_with_metadat = df_non_null.join(meta_data_dict["heuristic_metadata"])

In [44]:
df_with_metadat = df_with_metadat.reset_index().rename(columns={"index": "id"})

In [46]:
df_with_metadat.to_parquet("../datasets/local/vis_papers/processed/vis_papers.parquet")

In [53]:
word_long_format = pd.DataFrame(
    [(word, i) for i, doc in enumerate(words) for word in doc], columns=["word", "id"]
)

In [54]:
token_long_format = pd.DataFrame(
    [(token, i) for i, doc in enumerate(tokens) for token in doc],
    columns=["token", "id"],
)

In [58]:
token_long_format.to_parquet(
    "../datasets/local/vis_papers/processed/vis_papers_tokens.parquet"
)
word_long_format.to_parquet(
    "../datasets/local/vis_papers/processed/vis_papers_words.parquet"
)

In [60]:
meta_data_dict["model_embeddings"]["all-mpnet-base-v2"].shape

torch.Size([3549, 768])

In [63]:
import torch

In [64]:
torch.save(
    meta_data_dict["model_embeddings"]["all-mpnet-base-v2"],
    "../datasets/local/vis_papers/processed/vis_papers_embeddings.pt",
)

In [1]:
import pandas as pd

pd.read_parquet("../datasets/local/vis_papers/processed/vis_papers_tokens.parquet")

,token,id
0,b'Accessibility',0
1,b' guidelines',0
2,b' place',0
3,b' restrictions',0
4,b' on',0
...,...,...
689996,b'X',3548
689997,b'&gt',3548
689998,b';&',3548
689999,b'gt',3548
